# W&B Job Creation

Learn how to create reusable jobs from W&B, which can be executed on different ML environments using Launch. This can be useful for:

- Model evaluation — define suites of tests to run on promising new model variants
- Retraining and fine tuning — repeat training or fine tuning on larger hardware or multiple nodes
- Scaled out hyperparameter sweeps — use larger machines, and more of them, to accelerate hyperparameter tuning.


Here, we'll go through [several ways](https://docs.wandb.ai/ref/cli/wandb-job/wandb-job-create) to a create a reusable job, using a FMNIST training job. (See more example jobs [here](https://github.com/wandb/launch-jobs/tree/main/jobs) or at https://wandb.ai/wandb/jobs/jobs )

In this quickstart we will:
1. Create a job from a **local directory**.
1. Create a job from a **Github repo**.
1. Create a job from a **training run**.
1. Create a **queue**.
1. Start an **agent**.
1. **Launch** the jobs!

From here, you'll be able to see it all working in the GUI and take the next step to scaling your ML work on powerful infrastructure.

This report has been taken from wandb website


# Set up W&B to get started

In [1]:
# Install W&B
!pip install wandb==0.16.0 -qqq

In [2]:
# Log in to your free W&B Account
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: teimuri (graph_imbalance). Use `wandb login --relogin` to force relogin


True

In [3]:
WB_PROJECT = "job-creation-demo"
WB_ENTITY = "teimuri" # Add your entity

# 1) Create a Job from a **local directory**

Clone the example job repository.

In [4]:
!git clone https://github.com/wandb/launch-jobs.git

fatal: destination path 'launch-jobs' already exists and is not an empty directory.


Now run this command to create a job:




In [10]:
! wandb job create -p $WB_PROJECT -e $WB_ENTITY -n "fashion-mnist-train" code launch-jobs/jobs/fashion_mnist_train -E 'job.py'

wandb: Creating launch job of type: code...
wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Adding directory to artifact (./launch-jobs/jobs/fashion_mnist_train)... Done. 0.2s
wandb:                                                                                
wandb: W&B sync reduced upload amount by 28.1%             
wandb: Updated job: teimuri/job-creation-demo/fashion-mnist-train:v1, with alias: latest
wandb: View all jobs in project 'job-creation-demo' here: https://wandb.ai/teimuri/job-creation-demo/jobs
wandb: 


You now have a job created and ready to launch.  If you want to skip right to launching, go to step 4.  Otherwise, try two other ways to create jobs.

# 2) Create a job from a **Github repo**
When you run the command below, W&B will build a job from this public repo.

In [5]:
! wandb job create -p $WB_PROJECT -e $WB_ENTITY -n "First_Git_Job" git https://github.com/wandb/launch-jobs/tree/main/jobs/fashion_mnist_train -E 'job.py'

wandb: Creating launch job of type: git...
wandb: Using requirements.txt in /jobs/fashion_mnist_train
wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb:                                                                                
wandb: Created job: teimuri/job-creation-demo/First_Git_Job:v0, with alias: latest
wandb: View all jobs in project 'job-creation-demo' here: https://wandb.ai/teimuri/job-creation-demo/jobs
wandb: 


# 3) Create a job from **existing training code**

Next, we're going to create a job by logging the code to W&B.  We'll create a tiny sample Python script, which will be used to create a W&B job using `log_code()`.

In [1]:
demo_job = """
import wandb
import random

with wandb.init(project="fmnist_job_tutorial") as run:
  run.log({"eval_metric": random.random()})
  print("This job could run an Optuna Study automatically, for example.")
  run.log_code()
"""

with open('demo_job.py', 'w+') as file:
  file.write(demo_job)

requirements = """
wandb
random
"""

with open('requirements.txt', 'w+') as file:
  file.write(requirements)

Now run the training code.

In [7]:
!python demo_job.py

wandb: Currently logged in as: teimuri (graph_imbalance). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /home/taha/Documents/wss/wandb/run-20240227_123856-qmpiwsxn
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run jolly-voice-1
wandb: ⭐️ View project at https://wandb.ai/teimuri/fmnist_job_tutorial
wandb: 🚀 View run at https://wandb.ai/teimuri/fmnist_job_tutorial/runs/qmpiwsxn
This job could run an Optuna Study automatically, for example.
wandb:                                                                                
wandb: 
wandb: Run history:
wandb: eval_metric ▁
wandb: 
wandb: Run summary:
wandb: eval_metric 0.66285
wandb: 
wandb: 🚀 View run jolly-voice-1 at: https://wandb.ai/teimuri/fmnist_job_tutorial/runs/qmpiwsxn
wandb: ️⚡ View job at https://wandb.ai/teimuri/fmnist_job_tut

# 4) Create a Launch Queue

Now let's create a queue to run these Jobs on. Queues are our interface into ML infrastructure — W&B supports Kubernetes, Sagemaker, and Docker queues (the latter being most useful for individual workstations).


In [8]:
import json
config ={"label":"tutorial-test"}
api = wandb.Api()
queue = api.create_run_queue('tutorial-queue', 'local-container',access='project', entity=WB_ENTITY, config=config)

# 5) Start an Agent

The agent will pull down Jobs from the queue to run in Docker containers — install and start [Docker](https://www.docker.com/products/docker-desktop/) to use the agent on your laptop.


## 💻 `in your laptop terminal`
```shell
pip install wandb --upgrade
wandb login
wandb launch-agent -e <entity-name> -q tutorial-queue
```

# 6) Launch the job

Now we're all set to launch the job.  Run the command below, then look over at your terminal to see the job in progress.

In [11]:
! wandb launch -j "$WB_ENTITY/$WB_PROJECT/fashion-mnist-train:latest" -e <entity-name> -q tutorial-queue

wandb: launch: Launching run into teimuri/job-creation-demo
wandb: launch: Added run to queue tutorial-queue.
wandb: launch: Launch spec:
wandb: {'docker': {},
wandb:  'entity': 'teimuri',
wandb:  'git': {},
wandb:  'job': 'teimuri/job-creation-demo/fashion-mnist-train:latest',
wandb:  'overrides': {},
wandb:  'project': 'job-creation-demo',
wandb:  'resource': 'local-container',
wandb:  'resource_args': {'local-container': {'label': 'tutorial-test'}}}
wandb: 
wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb:   2 of 2 files downloaded.  
wandb:                                                                                


# You did it!

You registered a job and ran it using Launch! If you go to your project, you'll see a new run corresponding to the job you just ran.

To run a hyperparameter sweep on your queue--with condition configuration and multi-objective optimization--jump over to [this notebook](https://colab.research.google.com/drive/1WxLKaJlltThgZyhc7dcZhDQ6cjVQDfil#scrollTo=sHcr30CKybN7).  
